In [1]:
from pancakes import Pancakes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table

%matplotlib widget

In [2]:
tess_file = './PS1_to_TESS/data/tess2019359002923-s0020-1-1-0165-s_ffic.fits'
savepath = './SkyCells/Sector020/'

pancakes = Pancakes(tess_file, 
                    savepath = '/Users/zgl12/Python_Scripts/SynDiff/', 
                    num_cores = 1, sector=20, use_multiple_cores_per_task = False, overwrite = True)

Sector: 20


Getting all WCS: 100%|██████████| 1739/1739 [00:34<00:00, 50.40it/s]


In [3]:
pancakes.syrup()

rm: /Users/zgl12/Python_Scripts/SynDiff/TESS_s0020_1_skycell.2246.021_00002_.fits: No such file or directory


In [ ]:
ps1_filename = 'ds9.fits'
folder = '/Users/zgl12/Modules/SynDiff/development/PS1_to_TESS/data/'
tess_filename = 'tess2019359002923-s0020-1-1-0165-s_ffic.fits'
file = '/Users/zgl12/Python_Scripts/SynDiff/TESS_s0020_1_master_pixels2skycells.fits.gz'


tess_hdul = fits.open(folder + tess_filename)
tess_data = tess_hdul[1].data
tess_header = tess_hdul[1].header
tess_wcs = WCS(tess_header)
tess_hdul.close()

ps1_hdul = fits.open(folder + ps1_filename)
ps1_data = ps1_hdul[0].data
ps1_header = ps1_hdul[0].header
ps1_wcs = WCS(ps1_header)
ps1_hdul.close()

print(ps1_data.shape)

ps1_fprint = ps1_wcs.calc_footprint()
p_xs, p_ys = tess_wcs.all_world2pix(ps1_fprint[:, 0], ps1_fprint[:, 1], 0)


In [ ]:

hdul = fits.open(file)
data = hdul[1].data
table_data = hdul[2].data
hdul.close()

tab = Table(table_data).to_pandas()

tab_f = tab[tab['SKYCELL'] == 'skycell.2246.021']

print(tab_f)

# plt.figure(figsize=(12, 12))
fig, axs = plt.subplots(1, 2, sharex=False, sharey=False)
axs[0].imshow(data, origin='lower')
axs[0].scatter(p_xs, p_ys, color='red', s = 10)
axs[1].imshow(data, origin='lower')
axs[1].set_xlim(1000, 1200)
axs[1].set_ylim(500, 700)
plt.savefig('skycell_for_tess.png', dpi = 900)
plt.show()

# PANstarrs Coordinate and Astrometry for Key Equivalent Skycells

In [ ]:
# data
data[(data != 952) & (data != 951)] = 0

plt.figure()
plt.imshow(data, vmin = 950, vmax = 953)
# plt.scatter(t_xs, t_ys, c='r', s=10)
plt.scatter(p_xs, p_ys, c='r', s=10)

plt.plot([p_xs[0], p_xs[1]], [p_ys[0], p_ys[1]], c='r')
plt.plot([p_xs[1], p_xs[2]], [p_ys[1], p_ys[2]], c='r')
plt.plot([p_xs[2], p_xs[3]], [p_ys[2], p_ys[3]], c='r')
plt.plot([p_xs[0], p_xs[3]], [p_ys[0], p_ys[3]], c='r')

plt.xlim(950, 1125)
plt.ylim(950, 1125)
plt.savefig('buffer_zone.png', dpi = 900)
plt.show()
